<a href="https://colab.research.google.com/github/Lukewtr/Combinatorial-Decision-Making-Optimization/blob/main/SMT_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
# Install the required libraries:
!pip3 install z3-solver

In [90]:
# Import the required modules:
import argparse
import os

import numpy as np
import time
from z3 import *

In [134]:
optim = False   #TO BE CHANGED

covid_span = 14
interval_dates = 5

max_p = 31 
max_d = 25

np = 31         #TO BE CHANGED
nd = 25         #TO BE CHANGED

bias_p = 0      #TO BE CHANGED
bias_d = 0      #TO BE CHANGED

np_min = min(0+bias_p, max_p-np)
np_max = min(np+bias_p, max_p)

nd_min = min(0+bias_d, max_d-nd)
nd_max = min(nd+bias_d, max_d)

min_ppl = 5     #TO BE CHANGED
max_ppl = 8     #TO BE CHANGED

m = [[0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0],
 [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
 [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
 [0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1],
 [1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
 [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
 [0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0],
 [1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1],
 [1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1],
 [1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1],
 [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1],
 [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
 [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0]]

box_notBool = []
for i in range(np_min,np_max):
    box_notBool.append(m[i][nd_min:nd_max])

box = []
for i in range(len(box_notBool)):
    subbox = []
    for j in range(len(box_notBool[0])):
        subbox.append(True if box_notBool[i][j]==1 else False)
    box.append(subbox)

box = box_notBool
#print(box)

[[0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1], [0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1], [1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], [0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [135]:
# VARIABLES:----------------------------------------------------
# Matrix solution Y:
Y = []
for i in range(np):
    for j in range(nd):
        Y.append(Int("y_{}_{}".format(i,j)))
#print(Y)

# Optimal matrix solution Y:
OY = []
for i in range(np):
    for j in range(nd): pass
        #OY.append(Int("oy_{}_{}".format(i,j)))

# 3D co-partecipations matrix COY:
COY = []
for i in range(np):
    for j in range(nd):
        for oi in range(np):
            COY.append(Int("coy_{}_{}_{}".format(i,j,oi)))
#print(COY)            

# Cumulative partecipations matrix covid:
#covid = []
#for i in range(np):
#    for j in range(nd):
#        covid.append(Int("covid_{}_{}".format(i,j)))

# Date solution X:
X = []
for j in range(nd):
    X.append(Int("x_{}".format(j)))

# Cumulative Dates XX:
XX = []
for j in range(nd):
    XX.append(Int("xx_{}".format(j)))

# Optimal Dates XX:
OX = []
for j in range(nd): pass
    #OX.append(Int("ox_{}".format(j)))

# Partecipants solution I:
I = []
for i in range(np):
    I.append(Int("i_{}".format(i)))

# Cumulative Partecipants II:
II = []
for i in range(np):
    II.append(Int("ii_{}".format(i)))

[y_0_0, y_0_1, y_0_2, y_0_3, y_0_4, y_0_5, y_0_6, y_0_7, y_0_8, y_0_9, y_0_10, y_0_11, y_0_12, y_0_13, y_0_14, y_0_15, y_0_16, y_0_17, y_0_18, y_0_19, y_0_20, y_0_21, y_0_22, y_0_23, y_0_24, y_1_0, y_1_1, y_1_2, y_1_3, y_1_4, y_1_5, y_1_6, y_1_7, y_1_8, y_1_9, y_1_10, y_1_11, y_1_12, y_1_13, y_1_14, y_1_15, y_1_16, y_1_17, y_1_18, y_1_19, y_1_20, y_1_21, y_1_22, y_1_23, y_1_24, y_2_0, y_2_1, y_2_2, y_2_3, y_2_4, y_2_5, y_2_6, y_2_7, y_2_8, y_2_9, y_2_10, y_2_11, y_2_12, y_2_13, y_2_14, y_2_15, y_2_16, y_2_17, y_2_18, y_2_19, y_2_20, y_2_21, y_2_22, y_2_23, y_2_24, y_3_0, y_3_1, y_3_2, y_3_3, y_3_4, y_3_5, y_3_6, y_3_7, y_3_8, y_3_9, y_3_10, y_3_11, y_3_12, y_3_13, y_3_14, y_3_15, y_3_16, y_3_17, y_3_18, y_3_19, y_3_20, y_3_21, y_3_22, y_3_23, y_3_24, y_4_0, y_4_1, y_4_2, y_4_3, y_4_4, y_4_5, y_4_6, y_4_7, y_4_8, y_4_9, y_4_10, y_4_11, y_4_12, y_4_13, y_4_14, y_4_15, y_4_16, y_4_17, y_4_18, y_4_19, y_4_20, y_4_21, y_4_22, y_4_23, y_4_24, y_5_0, y_5_1, y_5_2, y_5_3, y_5_4, y_5_5, y_5_6, 

In [136]:
# CONSTRAINTS:---------------------------------------------------
if not optim: s = Solver()
else: s = Optimize()

# Y matrix:
for var in Y:
    s.add(Or(var==1, var==0))

# 0) Matrices consturction:
# OY matrix:
def f(i,oi):
    if i//2 == oi//2: return 1
    else: return 0
def g(i,oi):
    if i//3 == oi//3: return 1
    else: return 0
#for j in range(nd):
#    for i in range(np):
#        s.add(If(
#            Y[i*nd + j]==1,
#            OY[i*nd + j] == Sum([f(i,oi) + g(i,oi) for oi in range(np) if oi!=i]),
#            OY[i*nd + j] == 0
#        ))
        
# X matrix:
for j in range(nd):
    s.add(Or(X[j]==1, X[j]==0))

for j in range(nd):
    #s.add(If(Sum([box[i][j] for i in range(np)]) > min_ppl, 
    #         X[j]==1, 
    #         X[j]==0))
    s.add(If(X[j]==1, 
             XX[j] >= min_ppl, 
             XX[j] == 0))    
    
# XX matrix:
for j in range(nd):
    s.add(XX[j] == Sum([Y[i]for i in range(j, len(Y), nd)]))

# OX matrix:
for j in range(nd): pass
#    s.add(OX[j] == Sum([OY[i]for i in range(j, len(Y), nd)]))

# I matrix:
for i in range(np):
    s.add(Or(I[i]==1, I[i]==0))

for i in range(np):
    s.add(If(Sum(box[i]) > 0, 
             I[i]==1, 
             I[i]==0))
    s.add(If(I[i]==1, 
             Or([Y[j]==1 for j in range(i*nd, (i+1)*nd)]), 
             And([Y[j]==0 for j in range(i*nd, (i+1)*nd)])))    
    
# II matrix:
for i in range(np):
    s.add(II[i] == Sum([Y[j] for j in range(i*nd, (i+1)*nd)]))

# COY matrix:
def idx(i,j,oi):
    return i*np*nd + j*np + oi

for i in range(np):
    for j in range(nd):
        for oi in range(np):
            s.add(Or(COY[idx(i,j,oi)]==1, COY[idx(i,j,oi)]==0))

for i in range(np):
    for j in range(nd):
        for oi in range(np):
            if i==oi: s.add(COY[idx(i,j,oi)]==0)
            else:
                min_j = min(0, j - covid_span//interval_dates)
                max_j = min(nd, j + covid_span//interval_dates)

                s.add(If((
                    Or([And(Y[i*nd+jj]==1, 
                            Y[oi*nd+jj]==1) for jj in range(min_j, max_j)])
                ), COY[idx(i,j,oi)]==1, COY[idx(i,j,oi)]==0))


# 1) Check availability:
for i in range(np):
    for j in range(nd):
        #s.add(Or(Y[i*nd+j]==0, box[i][j]==1))
        s.add(If(box[i][j]==0, Y[i*nd + j]==0, True))


# 2) Partecipations are in range(min_ppl,max_ppl) per each date and every kid 
#     who has given at least one availability should come at least once:
for j in range(nd):
    s.add(If(X[j]==1, And(XX[j]>=min_ppl, XX[j]<=max_ppl), XX[j]==0))


# 3) Covid constraint:
for i in range(np):
    for j in range(nd):
        s.add(sum(COY[idx(i,j,0):idx(i,j+1,0)]) < max_ppl)

In [137]:
if optim: s.maximize(Sum(OX))
before = time.time()
print(s.check())
after = time.time()

sat


In [138]:
print(after-before)

603.3063988685608


In [139]:
m = s.model()

print([m[X[j]] for j in range(nd)])
print([m[XX[j]] for j in range(nd)], "\n")
print("Solutions: \t\t\t\t\t Given Availability:")
for i in range(np):
    print([m[Y[j]] for j in range(i*nd,(i+1)*nd)], "  ", m[II[i]], "\t", box[i])

[1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[5, 5, 5, 5, 0, 5, 5, 0, 0, 5, 0, 0, 0, 0, 0, 5, 0, 0, 5, 5, 0, 0, 5, 0, 0] 

Solutions: 					 Given Availability:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]    1 	 [0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]    2 	 [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]    1 	 [0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]    1 	 [1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]    2 	 [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 0, 0, 0, 0